# Pipeline for ORCA-CLEAN Deep Denoizing Network

**Requirements** - In order to run this notebook, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription - [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace with computer cluster - [Configure workspace](../../configuration.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../../README.md) - check the getting started section

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1 Import the required libraries

In [1]:
import datetime

from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

from azure.ai.ml import MLClient, Input, Output
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component
from azure.ai.ml.entities import AmlCompute, Environment, Model
from azure.ai.ml.constants import AssetTypes

## 1.2 Configure credential

We are using `DefaultAzureCredential` to get access to workspace. 
`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

Reference for more available credentials if it does not work for you: [configure credential example](../../configuration.ipynb), [azure-identity reference doc](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [2]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

## 1.3 Get a handle to the workspace

We use config file to connect to a workspace. The Azure ML workspace should be configured with computer cluster. [Check this notebook for configure a workspace](../../configuration.ipynb)

In [3]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

compute_instance_name = "whalesdenoisinggpu2"
try:
    print(ml_client.compute.get(compute_instance_name))
except Exception as ex:
    compute_instance = AmlCompute(
        name=compute_instance_name,
        type="amlcompute",
        size="Standard_NC24ads_A100_v4",
        # Minimum running nodes when there is no job running
        min_instances=0,
        # Nodes in cluster
        max_instances=4,
        # How many seconds will the node running after the job termination
        idle_time_before_scale_down=180,
        # Dedicated or LowPriority. The latter is cheaper but there is a chance of job termination
        tier="Dedicated",
    )
    ml_client.begin_create_or_update(compute_instance).result()
    print(f"Created new compute instance: {ml_client.compute.get(compute_instance_name)}")

Found the config file in: /config.json


enable_node_public_ip: true
id: /subscriptions/03fd01f6-6051-4545-a78e-ceaace399b96/resourceGroups/lianatests/providers/Microsoft.MachineLearningServices/workspaces/humpbackwhales-aml/computes/whalesdenoisinggpu2
idle_time_before_scale_down: 180
location: westeurope
max_instances: 4
min_instances: 0
name: whalesdenoisinggpu2
network_settings: {}
provisioning_state: Succeeded
size: STANDARD_NC24ADS_A100_V4
ssh_public_access_enabled: true
tier: dedicated
type: amlcompute



## 1.4 Create a custom environment

In [4]:
custom_env_name = "whales-denoising-env"
version = "1.3"
dependencies_dir = "./dependencies"

try:
    pipeline_job_env = ml_client.environments.get(name=custom_env_name, version=version)
    print(
        f"Environment with name {pipeline_job_env.name} is registered to workspace, the environment version is {pipeline_job_env.version}"
    )
except:
    pipeline_job_env = Environment(
        name=custom_env_name,
        description="Custom environment for running ORCA-CLEAN pipeline",
        conda_file=os.path.join(dependencies_dir, "conda.yaml"),
        image="mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.1-cudnn7-ubuntu18.04", #"mcr.microsoft.com/azureml/curated/acpt-pytorch-1.13-cuda11.7:18",
        version=version,
    )

    pipeline_job_env = ml_client.environments.create_or_update(pipeline_job_env)
    print(
        f"Environment with name {pipeline_job_env.name} is registered to workspace, the environment version is {pipeline_job_env.version}"
    )

Environment with name whales-denoising-env is registered to workspace, the environment version is 1.3


# 2. Define and create components into workspace
## 2.1 Load components from YAML

In [37]:
parent_dir = "."
train_model = load_component(source=parent_dir + "/train_model.yml")
#score_data = load_component(source=parent_dir + "/score_data.yml")
#eval_model = load_component(source=parent_dir + "/eval_model.yml")

## 2.2 Inspect loaded component

In [38]:
# Print the component as yaml
print(train_model)

$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: train_model
version: 0.0.1
display_name: Train Deep Denoising Model
description: The training component
type: command
inputs:
  whales_data:
    type: uri_folder
    mode: rw_mount
  noise_train_data:
    type: uri_folder
    mode: rw_mount
  noise_val_data:
    type: uri_folder
    mode: rw_mount
  noise_test_data:
    type: uri_folder
    mode: rw_mount
  max_train_epochs:
    type: integer
    default: '150'
  learning_rate:
    type: number
    default: '0.001'
  batch_size:
    type: integer
    default: '16'
  num_workers:
    type: integer
    default: '6'
  augmentation:
    type: integer
    default: '1'
  n_fft:
    type: integer
    default: '4096'
  hop_length:
    type: integer
    default: '441'
  sequence_len:
    type: integer
    default: '1280'
  freq_compression:
    type: string
    default: linear
  model_dir:
    type: uri_folder
    mode: rw_mount
  log_dir:
    type: uri_fold

# 3. Pipeline job
## 3.1 Build pipeline

In [39]:
# Construct pipeline
@pipeline()
def pipeline_with_components_from_yaml(
    whales_data_input,
    noise_train_data_input,
    noise_val_data_input,
    noise_test_data_input,
    model_dir_output,
    log_dir_output,
    checkpoint_dir_output,
    cache_dir_output,
    summary_dir_output,
    max_train_epochs,
    
):
    """Pipeline with components defined via yaml."""
    train_step = train_model(
        whales_data=whales_data_input,
        noise_train_data=noise_train_data_input,
        noise_val_data=noise_val_data_input,
        noise_test_data=noise_test_data_input,
        model_dir=model_dir_output,
        log_dir=log_dir_output,
        checkpoint_dir=checkpoint_dir_output,
        cache_dir=cache_dir_output,
        summary_dir=summary_dir_output,
        max_train_epochs=max_train_epochs
    )
    
    return {
        "trained_model": train_step.outputs.model_output
    }

mounted_dir = "/mnt/humpbackwhales/data/denoising_data"
mounter_output_dir = "/mnt/humpbackwhales/denoising/ML_pipeline"
#mounted_dir = azureml://datastores/workspaceblobstore/paths/tutorial-datasets/places2/train/

pipeline_job = pipeline_with_components_from_yaml(
    whales_data_input=Input(type="uri_folder", path=mounted_dir + "/whales_data_v2/"),
    noise_train_data_input=Input(type="uri_folder", path=mounted_dir + "/noise_train_v2/"),
    noise_val_data_input=Input(type="uri_folder", path=mounted_dir + "/noise_val_v2/"),
    noise_test_data_input=Input(type="uri_folder", path=mounted_dir + "/noise_test_v2/"),
    model_dir_output=Input(type="uri_folder", path=mounter_output_dir + "/models/"),
    log_dir_output=Input(type="uri_folder", path=mounter_output_dir + "/logs/"),
    checkpoint_dir_output=Input(type="uri_folder", path=mounter_output_dir + "/model_checkpoints/"),
    cache_dir_output=Input(type="uri_folder", path=mounter_output_dir + "/cache/"),
    summary_dir_output=Input(type="uri_folder", path=mounter_output_dir + "/summary/"),
    max_train_epochs=5,
)

# set pipeline level compute
pipeline_job.settings.default_compute = compute_instance_name

In [40]:
# Inspect built pipeline
print(pipeline_job)

display_name: pipeline_with_components_from_yaml
description: Pipeline with components defined via yaml.
type: pipeline
inputs:
  whales_data_input:
    type: uri_folder
    path: azureml:/mnt/humpbackwhales/data/denoising_data/whales_data_v2/
  noise_train_data_input:
    type: uri_folder
    path: azureml:/mnt/humpbackwhales/data/denoising_data/noise_train_v2/
  noise_val_data_input:
    type: uri_folder
    path: azureml:/mnt/humpbackwhales/data/denoising_data/noise_val_v2/
  noise_test_data_input:
    type: uri_folder
    path: azureml:/mnt/humpbackwhales/data/denoising_data/noise_test_v2/
  model_dir_output:
    type: uri_folder
    path: azureml:/mnt/humpbackwhales/denoising/ML_pipeline/models/
  log_dir_output:
    type: uri_folder
    path: azureml:/mnt/humpbackwhales/denoising/ML_pipeline/logs/
  checkpoint_dir_output:
    type: uri_folder
    path: azureml:/mnt/humpbackwhales/denoising/ML_pipeline/model_checkpoints/
  cache_dir_output:
    type: uri_folder
    path: azureml:/

## 3.2 Submit pipeline job

In [41]:
# Submit pipeline job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="whales_denoising"
)
pipeline_job

Your file exceeds 100 MB. If you experience low speeds, latency, or broken connections, we recommend using the AzCopyv10 tool for this file transfer.

Example: azcopy copy '/mnt/humpbackwhales/data/denoising_data/whales_data_v2/' 'https://humpbackwhalessa.blob.core.windows.net/azureml-blobstore-f4e80658-2a40-4426-8676-c5538afa6a8f/LocalUpload/c4f8edf663ec11fe0d34c4808ef63531/whales_data_v2' 

See https://docs.microsoft.com/azure/storage/common/storage-use-azcopy-v10 for more information.
Your file exceeds 100 MB. If you experience low speeds, latency, or broken connections, we recommend using the AzCopyv10 tool for this file transfer.

Example: azcopy copy '/mnt/humpbackwhales/data/denoising_data/noise_train_v2/' 'https://humpbackwhalessa.blob.core.windows.net/azureml-blobstore-f4e80658-2a40-4426-8676-c5538afa6a8f/LocalUpload/2a9e34de5512ddd07b23092d3f2dd598/noise_train_v2' 

See https://docs.microsoft.com/azure/storage/common/storage-use-azcopy-v10 for more information.


Experiment,Name,Type,Status,Details Page
whales_denoising,jovial_steelpan_gcmh21z3s2,pipeline,Preparing,Link to Azure Machine Learning studio


In [ ]:
# Wait until the job completes
ml_client.jobs.stream(pipeline_job.name)

## 3.3 Register a model

In [7]:
run_model = Model(
    path="azureml://jobs/{}/outputs/artifacts/paths/model/".format(pipeline_job.name),
    name="orca-clean-model-v1",
    description="Model trained with 5 epochs, whales_data_v2 and noise_train/test/val_v2.",
    type=AssetTypes.MLFLOW_MODEL
)

ml_client.models.create_or_update(run_model)

HttpResponseError: (UserError) The request is invalid.
Code: UserError
Message: The request is invalid.
Exception Details:	(NoMatchingArtifactsFoundFromJob) No artifacts matching model found from Job.
	Code: NoMatchingArtifactsFoundFromJob
	Message: No artifacts matching model found from Job.
Additional Information:Type: ComponentName
Info: {
    "value": "managementfrontend"
}Type: Correlation
Info: {
    "value": {
        "operation": "684650c9348bd0465b71ae6d4ce830e1",
        "request": "7df8c33f08f3e5ed"
    }
}Type: Environment
Info: {
    "value": "westeurope"
}Type: Location
Info: {
    "value": "westeurope"
}Type: Time
Info: {
    "value": "2023-09-15T09:06:04.6361655+00:00"
}

## 3.4 Create a batch endpoint

In [ ]:
from azure.ai.ml.entities import BatchEndpoint, ModelBatchDeployment, PipelineComponentBatchDeployment

endpoint_name="whales-denoising-batch"

endpoint = BatchEndpoint(
    name=endpoint_name,
    description="Endpoint for pipeline deployments",
)

ml_client.batch_endpoints.begin_create_or_update(endpoint).result()

endpoint = ml_client.batch_endpoints.get(name=endpoint_name)
print(endpoint)

## 3.5 Deploy the pipeline job

In [ ]:
deployment = PipelineComponentBatchDeployment(
    name="whales-denoising-batch-from-job",
    description="This deployment is created from a pipeline job.",
    endpoint_name=endpoint.name,
    job_definition=pipeline_job,
    settings={
        "default_compute": compute_instance_name,
        "continue_on_step_failure": False
    }
)

In [ ]:
ml_client.batch_deployments.begin_create_or_update(deployment).result()

In [ ]:
endpoint = ml_client.batch_endpoints.get(endpoint.name)
endpoint.defaults.deployment_name = deployment.name
ml_client.batch_endpoints.begin_create_or_update(endpoint).result()

## 3.6 Test the deployment

In [ ]:
job = ml_client.batch_endpoints.invoke(
    endpoint_name=endpoint.name, 
)

In [ ]:
ml_client.jobs.get(name=job.name)